# Quantum Algorithms Demo Notebook

This notebook demonstrates the core quantum algorithms using an abstract hamiltonian interface: GQSP, Trotter decomposition, and Preparation-Selection methods using numpy and vanilla Python.

## Table of Contents

1. [Setup and Test Environment](#setup)
2. [GQSP Algorithm - Basic vs Multi-Depth](#gqsp)
3. [Trotter Decomposition - Two-Hamiltonian vs Multi-Hamiltonian](#trotter)
4. [Preparation-Selection - Matrix vs Operator Chain](#prepsel)
5. [Complete integration](#integration)
6. [Amplitude Amplification and Grovers](#Grover)



---

## 1. Setup and Test Environment <a name='setup'></a>

In [ ]:
import numpy as np
from itertools import combinations
import pyqasm as pq
import string

# Import quantum algorithm libraries
from qbraid_algorithms.evolution import GQSP, Trotter, create_test_hamiltonians
from qbraid_algorithms.embedding import PauliOperator, Prep, PrepSelLibrary, Select
from qbraid_algorithms.qtran import QasmBuilder, std_gates, GateLibrary, GateBuilder
from qbraid_algorithms.amplitude_amplification import AALibrary
np.set_printoptions(linewidth=np.inf,precision=2,suppress=True)

In [3]:
print("Quantum Algorithms Demo",'\n',"=" * 40)
print("Testing GQSP, Trotter, and Prep-Select algorithms",'\n',"=" * 40)

# Initialize test environment
test_hamiltonians = create_test_hamiltonians(reg_size=3)
test_qubits = [*range(3)]

print(f"Created {len(test_hamiltonians)} test Hamiltonians")
print(f"Available Hamiltonians: {list(test_hamiltonians.keys())}")

Quantum Algorithms Demo 
Testing GQSP, Trotter, and Prep-Select algorithms 
Created 5 test Hamiltonians
Available Hamiltonians: ['tfim', 'heisenberg', 'random_dense', 'random_sparse', 'hubbard']



---

## 2. GQSP Algorithm Demonstrations <a name='gqsp'></a>

### Demo 2.1: Basic GQSP vs Variable Depth

In [4]:

def demo_gqsp_configurations():
    """Compare basic GQSP with different depth configurations."""
    print("\nGQSP Algorithm - Basic vs Multi-Depth")
    print("-" * 42)
    
    # Get a test Hamiltonian
    hamiltonian = list(test_hamiltonians.values())[0]
    
    # Configuration 1: Basic GQSP (depth=1)
    print("Configuration 1: Basic GQSP (depth=1)")
    basic_phases = [0.1, 0.2, 0.3]  # 2*1 + 1 = 3 phases

    # Setup qasm program
    builder1 = QasmBuilder(3)
    std1 = builder1.import_library(std_gates)
    gqsp1 = builder1.import_library(GQSP)

    # Hamiltonian abstraction interface (need to remove time evolution for GQSP)
    class BasicHam(hamiltonian):
        def apply(self, *args, **kwargs):
            super().apply(0.1, *args, **kwargs)
        def controlled(self, *args, **kwargs):
            super().controlled(0.1, *args, **kwargs)
    
    try:
        # Apply GQSP with basic Hamiltonian
        gqsp1.GQSP(test_qubits, basic_phases, BasicHam, depth=1)
        std1.measure(test_qubits, test_qubits)
        basic_program = builder1.build()
        
        print(f"Basic GQSP: {len(basic_program)} characters")
        print(f"\tPhases used: {len(basic_phases)}")
        print(basic_program)
        
    except Exception as e:
        print(f"Basic GQSP failed: {str(e)}")
        return
    
    # Configuration 2: Multi-depth GQSP (depth=3)
    print("\nConfiguration 2: Multi-depth GQSP (depth=3)")
    multi_phases = [0.1, 0.2, 0.3, 0.15, 0.25, 0.35, 0.05]  # 2*3 + 1 = 7 phases
    
    builder2 = QasmBuilder(3)
    std2 = builder2.import_library(std_gates)
    gqsp2 = builder2.import_library(GQSP)
    
    class MultiHam(hamiltonian):
        def apply(self, *args, **kwargs):
            super().apply(0.1, *args, **kwargs)
        def controlled(self, *args, **kwargs):
            super().controlled(0.1, *args, **kwargs)
    
    try:
        gqsp2.GQSP(test_qubits, multi_phases, MultiHam, depth=3)
        std2.measure(test_qubits, test_qubits)
        multi_program = builder2.build()
        
        print(f"Multi-depth GQSP: {len(multi_program)} characters")
        print(f"\tPhases used: {len(multi_phases)}")
        print(multi_program)
        
    except Exception as e:
        print(f"Multi-depth GQSP failed: {str(e)}")
        return
    
    # Compare configurations
    print(f"\nComparison:")
    print(f"   Basic (depth=1): {len(basic_program)} chars, {len(basic_phases)} phases")
    print(f"   Multi (depth=3): {len(multi_program)} chars, {len(multi_phases)} phases")
    print(f"   Size ratio: {len(multi_program) / len(basic_program):.2f}x")
    
    return {
        'basic': {'length': len(basic_program), 'phases': len(basic_phases)},
        'multi': {'length': len(multi_program), 'phases': len(multi_phases)}
    }

# Run GQSP configurations demo
gqsp_results = demo_gqsp_configurations()


GQSP Algorithm - Basic vs Multi-Depth
------------------------------------------
Configuration 1: Basic GQSP (depth=1)
Basic GQSP: 564 characters
	Phases used: 3
OPENQASM 3;
include "stdgates.inc";
qubit[4] qb;
bit[4] cb;
gate TFIM_3q_J100_h70(time) aa,ab,ac{
	cnot  aa, ab;
	rz(2.0 * time)  ab;
	cnot  aa, ab;
	cnot  ab, ac;
	rz(2.0 * time)  ac;
	cnot  ab, ac;
	cnot  ac, aa;
	rz(2.0 * time)  aa;
	cnot  ac, aa;
	rx(1.4 * time)  aa;
	rx(1.4 * time)  ab;
	rx(1.4 * time)  ac;
}

gate GQSP_1_TFIM(θa,θb,θc) aa,ab,ac,ad{
	ry(θa)  aa;
	ctrl(1) @ TFIM_3q_J100_h70(0.1)  aa, ab, ac, ad;
	p(θb)  aa;
	ry(θc)  aa;
}

GQSP_1_TFIM(0.1,0.2,0.3) qb[3],qb[0],qb[1],qb[2];
cb[{3}] = measure qb[{3}];
cb[{0, 1, 2}] = measure qb[{0, 1, 2}];


Configuration 2: Multi-depth GQSP (depth=3)
Multi-depth GQSP: 746 characters
	Phases used: 7
OPENQASM 3;
include "stdgates.inc";
qubit[4] qb;
bit[4] cb;
gate TFIM_3q_J100_h70(time) aa,ab,ac{
	cnot  aa, ab;
	rz(2.0 * time)  ab;
	cnot  aa, ab;
	cnot  ab, ac;
	rz(2.0 * time


---

## 3. Trotter Decomposition Examples <a name='trotter'></a>

### Demo 3.1: Two-Hamiltonian vs Multi-Hamiltonian Trotter

In [5]:
def demo_trotter_configurations():
    """Compare two-Hamiltonian and multi-Hamiltonian Trotter decomposition."""
    print("\nTrotter Decomposition - Two vs Multi-Hamiltonian")
    print("-" * 52)
    
    hamiltonians = list(test_hamiltonians.values())
    
    # Configuration 1: Two-Hamiltonian Trotter
    print("Configuration 1: Two-Hamiltonian Trotter (Suzuki)")
    ham1, ham2 = hamiltonians[0], hamiltonians[1]
    
    builder1 = QasmBuilder(3)
    std1 = builder1.import_library(std_gates)
    trotter1 = builder1.import_library(Trotter)
    
    try:
        trotter1.trot_suz(test_qubits, "0.5", ham1, ham2, depth=2)
        std1.measure(test_qubits, test_qubits)
        two_ham_program = builder1.build()
        
        print(f"Two-Hamiltonian: {len(two_ham_program)} characters")
        print(f"\tMethod: Suzuki-Trotter, Depth: 2")
        print(two_ham_program)
        
    except Exception as e:
        print(f"Two-Hamiltonian Trotter failed: {str(e)}")
        return
    
    # Configuration 2: Multi-Hamiltonian Trotter
    print("\nConfiguration 2: Multi-Hamiltonian Trotter")
    multi_hams = hamiltonians[:3]  # Use 3 Hamiltonians
    
    builder2 = QasmBuilder(3)
    std2 = builder2.import_library(std_gates)
    trotter2 = builder2.import_library(Trotter)
    
    try:
        trotter2.multi_trot_suz(test_qubits, "0.4", multi_hams, depth=2)
        std2.measure(test_qubits, test_qubits)
        multi_ham_program = builder2.build()
        
        print(f"Multi-Hamiltonian: {len(multi_ham_program)} characters")
        print(f"\tHamiltonians: 3, Depth: 2")
        # print(multi_ham_program)

    except Exception as e:
        print(f"Multi-Hamiltonian Trotter failed: {str(e)}")
        return
    
    # Configuration 3: Linear Trotter (bonus comparison)
    print("\nConfiguration 3: Linear Trotter Decomposition")
    
    builder3 = QasmBuilder(3)
    std3 = builder3.import_library(std_gates)
    trotter3 = builder3.import_library(Trotter)
    
    try:
        trotter3.trot_linear(test_qubits, "0.2", hamiltonians[:2], steps=4)
        std3.measure(test_qubits, test_qubits)
        linear_program = builder3.build()
        
        print(f"Linear Trotter: {len(linear_program)} characters")
        print(f"Method: First-order, Steps: 4")
        
    except Exception as e:
        print(f"Linear Trotter failed: {str(e)}")
        linear_program = ""
    
    # Compare all configurations
    print(f"Trotter Configuration Comparison:")
    print(f"\tTwo-Hamiltonian (Suzuki): {len(two_ham_program)} chars")
    print(f"\tMulti-Hamiltonian:       {len(multi_ham_program)} chars")
    if linear_program:
        print(f"\tLinear decomposition:     {len(linear_program)} chars")

    return {
        'two_ham': len(two_ham_program),
        'multi_ham': len(multi_ham_program),
        'linear': len(linear_program) if linear_program else 0
    }

# Run Trotter configurations demo
trotter_results = demo_trotter_configurations()


Trotter Decomposition - Two vs Multi-Hamiltonian
----------------------------------------------------
Configuration 1: Two-Hamiltonian Trotter (Suzuki)
Two-Hamiltonian: 1858 characters
	Method: Suzuki-Trotter, Depth: 2
OPENQASM 3;
include "stdgates.inc";
qubit[3] qb;
bit[3] cb;
gate TFIM_3q_J100_h70(time) aa,ab,ac{
	cnot  aa, ab;
	rz(2.0 * time)  ab;
	cnot  aa, ab;
	cnot  ab, ac;
	rz(2.0 * time)  ac;
	cnot  ab, ac;
	cnot  ac, aa;
	rz(2.0 * time)  aa;
	cnot  ac, aa;
	rx(1.4 * time)  aa;
	rx(1.4 * time)  ab;
	rx(1.4 * time)  ac;
}

gate HeisenbergXYZ_3q_Jx100_Jy120_Jz80(time) aa,ab,ac{
	ry(pi/2)  aa;
	ry(pi/2)  ab;
	cnot  aa, ab;
	rz(2.0 * time)  ab;
	cnot  aa, ab;
	ry(-pi/2)  aa;
	ry(-pi/2)  ab;
	rx(-pi/2)  aa;
	rx(-pi/2)  ab;
	cnot  aa, ab;
	rz(2.4 * time)  ab;
	cnot  aa, ab;
	rx(pi/2)  aa;
	rx(pi/2)  ab;
	cnot  aa, ab;
	rz(1.6 * time)  ab;
	cnot  aa, ab;
	ry(pi/2)  ab;
	ry(pi/2)  ac;
	cnot  ab, ac;
	rz(2.0 * time)  ac;
	cnot  ab, ac;
	ry(-pi/2)  ab;
	ry(-pi/2)  ac;
	rx(-pi/2)  ab;
	r


---

## 4. Preparation-Selection Algorithms <a name='prepsel'></a>

### Demo 4.1: Matrix Input vs Operator Chain Input

In [6]:

def demo_prep_select_configurations():
    """Compare prep-select with matrix input vs operator chain input."""
    print("\nPreparation-Selection - Matrix vs Operator Chain")
    print("-" * 52)
    
    test_qubits = [*range(4)]
    
    # Configuration 1: Matrix Input
    print("Configuration 1: Matrix Input")
    test_matrices = [
        ("Pauli-Z", np.array([[1, 0], [0, -1]])),
        ("Random 4x4", np.random.random((4, 4)) + 1j * np.random.random((4, 4)))
    ]
    
    matrix_results = {}
    
    for name, matrix in test_matrices:
        print(f"Testing {name} matrix {matrix.shape}...")
        
        builder = QasmBuilder(3)
        std = builder.import_library(std_gates)
        prep_sel = builder.import_library(PrepSelLibrary)
        
        try:
            prep_sel.prep_select(test_qubits, matrix, approximate=0.1)
            std.measure(test_qubits, test_qubits)
            
            program = builder.build()
            matrix_results[name] = len(program)
            
            print(f"{name}: {len(program)} characters")
            
        except Exception as e:
            matrix_results[name] = 0
            print(f"{name}: {str(e)}")
    
    # Configuration 2: Operator Chain Input
    print("\nConfiguration 2: Operator Chain Input")
    test_chains = [
        ("Single Pauli", [("X", 0.5), ("Z", 0.3), ("Y", 0.2)]),
        ("Two-qubit Pauli", [("XX", 0.7), ("ZZ", 0.4), ("XY", 0.1)])
    ]
    
    chain_results = {}
    
    for name, chain in test_chains:
        print(f"Testing {name} chain ({len(chain)} operators)...")
        
        builder = QasmBuilder(3)
        std = builder.import_library(std_gates)
        prep_sel = builder.import_library(PrepSelLibrary)
        
        try:
            prep_sel.prep_select(test_qubits[:2], chain)
            std.measure(test_qubits, test_qubits)
            
            program = builder.build()
            chain_results[name] = len(program)
            
            operators = [op for op, _ in chain]
            print(f"\t{name}: {len(program)} characters")
            print(f"\tOperators: {operators}")
            if name== "Single Pauli":
                print(program)

        except Exception as e:
            chain_results[name] = 0
            print(f"{name}: {str(e)}")
    
    # Compare configurations
    print(f"\nPrep-Select Configuration Comparison:")
    print(f"Matrix inputs:")
    for name, length in matrix_results.items():
        print(f"\t{name}: {length} chars")
    print(f"Operator chains:")
    for name, length in chain_results.items():
        print(f"\t{name}: {length} chars")

    return {'matrix': matrix_results, 'chain': chain_results}

# Run prep-select configurations demo
prep_select_results = demo_prep_select_configurations()


Preparation-Selection - Matrix vs Operator Chain
----------------------------------------------------
Configuration 1: Matrix Input
Testing Pauli-Z matrix (2, 2)...
[('Z', np.complex128(1+0j))]
Pauli-Z: 508 characters
Testing Random 4x4 matrix (4, 4)...
[('II', np.complex128(0.4604637367120007+0.838946781387169j)), ('XI', np.complex128(0.5655598377382944+0.588833075307832j)), ('XX', np.complex128(0.4238844968885312+0.3326317061016123j)), ('IX', np.complex128(0.33106499350145013+0.33007330113196753j)), ('XY', np.complex128(-0.06002262495215413+0.2992097507618375j)), ('IY', np.complex128(-0.1797796889495182+0.20356340119461624j)), ('YI', np.complex128(-0.13521719508002197+0.19794667244217146j)), ('ZX', np.complex128(0.02442078075316162-0.23360780554666702j)), ('XZ', np.complex128(-0.18057751029366081-0.148700172760367j)), ('ZY', np.complex128(0.2184605953247674+0.03821255057383305j)), ('IZ', np.complex128(-0.095147349933698+0.07351264181838879j)), ('ZI', np.complex128(-0.116164132271550

Random 4x4: 2019 characters

Configuration 2: Operator Chain Input
Testing Single Pauli chain (3 operators)...
[('X', 0.5), ('Z', 0.3), ('Y', 0.2)]
	Single Pauli: 719 characters
	Operators: ['X', 'Z', 'Y']
OPENQASM 3;
include "stdgates.inc";
qubit[5] qb;
bit[5] cb;
gate PREP_3905172865891942725 aa,ab{
	ry(1.0808368368267307) aa;
	ry(0.6608524809625362) ab;
	cry(-0.5404152269046208) aa,ab;
	cry(-0.540415208375584) aa,ab;
}

gate X aa{
	x aa;
}

gate Z aa{
	z aa;
}

gate Y aa{
	y aa;
}

gate SEL_8998145479025848162 aa,ab,ac,ad{
	ctrl(2) @ X aa,ab,ac,ad;
	x aa;
	ctrl(2) @ Z aa,ab,ac,ad;
	x ab;
	ctrl(2) @ Y aa,ab,ac,ad;
	x aa;
	x ab;
}

gate PS_2_8654309695166846104 aa,ab,ac,ad{
	PREP_3905172865891942725 aa,ab;
	SEL_8998145479025848162 aa,ab,ac,ad;
	inv @ PREP_3905172865891942725 aa,ab;
}

PS_2_8654309695166846104 qb[3],qb[4],qb[0],qb[1];
cb[{3, 4}] = measure qb[{3, 4}];
cb[{0, 1, 2, 3}] = measure qb[{0, 1, 2, 3}];

Testing Two-qubit Pauli chain (3 operators)...
[('XX', 0.7), ('ZZ', 0.4), 


---

## 5. Algorithm Integration Example <a name='integration'></a>

### Demo 5.1: Combined Algorithm Pipeline


In [ ]:

def demo_algorithm_integration():
    """Demonstrate combining multiple algorithms in a single quantum program."""
    print("\n🔗 Algorithm Integration - Combined Pipeline")
    print("-" * 44)
    
    hamiltonians = list(test_hamiltonians.values())[:2]
    
    print("Building combined algorithm pipeline...")
    print("Pipeline: Trotter → GQSP → Prep-Select")
    
    builder = QasmBuilder(8)
    qubits = [*range(8)]
    std = builder.import_library(std_gates)
    gqsp = builder.import_library(GQSP)
    trotter = builder.import_library(Trotter)
    prep_sel = builder.import_library(PrepSelLibrary)
    
    class IntegratedHam(hamiltonians[0]):
        def apply(self, *args, **kwargs):
            super().apply(0.1, *args, **kwargs)
        def controlled(self, *args, **kwargs):
            super().controlled(0.1, *args, **kwargs)
    
    try:
        # Step 1: Apply Trotter decomposition
        print("  Step 1: Applying Trotter decomposition...")
        trotter.trot_suz(qubits[:3], "0.1", hamiltonians[0], hamiltonians[1], depth=1)
        
        # Step 2: Apply GQSP
        print("  Step 2: Applying GQSP...")
        gqsp.GQSP(qubits[3:6], [0.1, 0.2, 0.3], IntegratedHam, depth=1)
        
        # Step 3: Apply prep-select
        print("  Step 3: Applying prep-select...")
        test_matrix = np.array([[1, 0], [0, -1]])  # Pauli-Z
        prep_sel.prep_select(qubits[6:], test_matrix)
        
        # Measure all qubits
        std.measure(qubits, qubits)
        
        # Build complete program
        integrated_program = builder.build()
        
        print(f"Integrated pipeline: {len(integrated_program)} characters")
        print(f"\tContains Trotter: {'trot_suz' in integrated_program}")
        print(f"\tContains GQSP: {'GQSP' in integrated_program or 'gqsp' in integrated_program.lower()}")
        print(f"\tContains PrepSelect: {'PS_' in integrated_program or 'prep' in integrated_program.lower()}")
        
        return {
            'success': True,
            'total_length': len(integrated_program),
            'algorithms_used': 3
        }
        
    except Exception as e:
        print(f"Algorithm integration failed: {str(e)}")
        return {'success': False, 'error': str(e)}

# Run integration demo
integration_results = demo_algorithm_integration()


🔗 Algorithm Integration - Combined Pipeline
--------------------------------------------
Building combined algorithm pipeline...
Pipeline: Trotter → GQSP → Prep-Select
  Step 1: Applying Trotter decomposition...
  Step 2: Applying GQSP...
  Step 3: Applying prep-select...
[('Z', np.complex128(1+0j))]
Integrated pipeline: 2465 characters
   Contains Trotter: True
   Contains GQSP: True
   Contains PrepSelect: True



---

## 6. Amplitude Amplification Example <a name='grover'></a>

### Demo 6.1: Grovers


In [16]:
# Create 3-qubit circuit with OpenQASM 3.0
alg = QasmBuilder(3, 0, version="3")
reg = list(range(3))

# Import standard gates and algorithm libraries
program = alg.import_library(std_gates)
ampl = alg.import_library(AALibrary)


class Za(GateLibrary):
    """Custom gate: controlled-Z on all qubits except index 2."""
    name = "Z_on_two"
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.name = f"Z_on_two{len(reg)}"
        names = string.ascii_letters
        qargs = [
            names[i // len(names)] + names[i % len(names)] for i in range(len(reg))
        ]

        sys = GateBuilder()
        std = sys.import_library(std_gates)
        std.call_space = " {}"

        ind = dict(zip(range(len(reg)), qargs))
        ind.pop(2)

        # Gate definition
        std.begin_gate(self.name, qargs)
        std.x(qargs[2])
        std.controlled_op("z", (qargs[2], list(ind.values())), n=len(reg) - 1)
        std.x(qargs[2])
        std.end_gate()

        # Collect gate definitions and imports
        self.merge(*sys.build(),self.name)

    def apply(self, qubits):
        """Apply the custom gate to a set of qubits."""
        self.call_gate(self.name, qubits[-1], qubits[:-1])

    def controlled(self, qubits, control):
        """Controlled version of the custom gate."""
        self.controlled_op(self.name, (qubits[-1], [control] + qubits[:-1]))


# Define input parameter
theta = program.add_var("theta", type="input angle[32]")

# Apply Grover with custom gate
ampl.grover(Za, reg, 3)

# Build and validate program
prog = alg.build()
print(prog)

# res = pq.loads(prog)
# print(res)


OPENQASM 3;
include "stdgates.inc";
qubit[3] qb;
gate Z_on_two3 aa,ab,ac{
	x  ac;
	ctrl(2) @ z  aa, ab, ac;
	x  ac;
}

def Grover3Z_on_two3(qubit[3] reg) {
	h  reg;
	for int i in [0:2] {
		//Za
		Z_on_two3  reg[0], reg[1], reg[2];
		h  reg;
		//Z0
		x  reg;
		ctrl(2) @ z  reg[0], reg[1], reg[0];
		x  reg;
		h  reg;
	}
}

input angle[32] theta ;
Grover3Z_on_two3(qb[{0 ,1 ,2}]);

